<br>
    <center>МИНИСТЕРСТВО НАУКИ И ВЫСШЕГО ОБРАЗОВАНИЯ</center>
    <center>федеральное государственное автономное образовательное учреждение высшего образования </center> <center>«Самарский национальный исследовательский университет имени академика С.П. Королева»</center>
    <center>(Самарский университет)</center> </br>

<br/>

<br>
<center>Институт 	     информатики и кибернетики</center>                                                   	  
<center>Кафедра 	     технической кибернетики</center>                                                              	
</br>
<br/>

<br>

<br/>
<br/>
<br/>
<br>
<center>ОТЧЕТ</center>
<center>по лабораторной работе №1</center>

<center>«Введение в MapReduce модель на Python»</center>
<br/>
<center>по дисциплине <strong>«Большие данные»</strong></center>
<br/>
<center></center>
</br>
<br/>
<br/>
<br/>

<br>
<p style="text-align:right;">Обучающийся: Скляров Д.В.
<br>6131-010402D
<br>    
<br>Преподаватель: Попов С.Б.
</p>
<br/>
<br/>
<br/>
    <br/>
<br/>
<br/>
<center>Самара 2024</center>


# Введение в MapReduce модель на Python


In [4]:
from typing import (
    NamedTuple,
    Iterator,
    Any,
)
from random import randint
import os
print(os.getcwd())

/home/nia/Desktop/ssau/big_data/lab1


In [5]:
MAX_VAL = 100
ARR_SIZE = 100
input_vals = [randint(0, MAX_VAL - 1) for _ in range(ARR_SIZE)]

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.


In [6]:
from typing import Any

iv = input_vals.copy()


class CTX:
    CTX_STORAGE = {}

    def __setattr__(self, name: str, value: Any) -> None:
        self.CTX_STORAGE[name] = value

    def __getattr__(self, attr: str) -> Any:
        try:
            return self.CTX_STORAGE[attr]
        except KeyError:
            return None


def ctx_inject(func):
    ctx = CTX()

    def wrapper(*args, **kwargs):
        return func(ctx, *args, **kwargs)
    return wrapper


@ctx_inject
def MAP(ctx: CTX, a):
    if ctx.max_value == None or ctx.max_value <= a:
        ctx.max_value = a
        return a


@ctx_inject
def REDUCE(ctx: CTX, val):
    if val is not None and ctx.max_value == val:
        yield val


def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


max_value,  = list(flatten(list(map(REDUCE, (map(MAP, iv))))))
assert max_value == max(iv)

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [7]:
import statistics


@ctx_inject
def combine(ctx, value):
    s, c = value
    try:
        ctx.sum += s
        ctx.cnt += c
    except TypeError:
        ctx.sum = s
        ctx.cnt = c
    return (ctx.sum, ctx.cnt)


def REDUCE(value):
    s, c = value
    return s / c


map_reduce_avg = list(
    map(REDUCE, (map(combine, (map(lambda x: (x, 1), input_vals))))))[-1]
assert map_reduce_avg == statistics.mean(input_vals)

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах


In [8]:
from typing import NamedTuple  # requires python 3.6+
from pprint import pprint


class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str


input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]


@ctx_inject
def clear_ctx(ctx):
    ctx.agreagregation = {}


@ctx_inject
def REDUCE(ctx, value):
    key, agg = value
    if ctx.agreagregation is None:
        ctx.agreagregation = {}

    aggregs = ctx.agreagregation.get(key, [])
    aggregs.append(agg)
    ctx.agreagregation[key] = aggregs
    return ctx.agreagregation


KEY = 'gender'
clear_ctx()
pprint(list(map(REDUCE, list(
    map(lambda x: (x.__getattribute__(KEY), x), input_collection))))[-1])

{'female': [User(id=1, age=25, social_contacts=240, gender='female'),
            User(id=2, age=25, social_contacts=500, gender='female'),
            User(id=3, age=33, social_contacts=800, gender='female')],
 'male': [User(id=0, age=55, social_contacts=20, gender='male')]}


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов


In [9]:
@ctx_inject
def map_phase_1(ctx, value: User):
    if ctx.existing_pairs is None:
        ctx.existing_pairs = []

    for field in value._fields:
        par = (field, value.__getattribute__(field))
        if par not in ctx.existing_pairs:
            yield par[0]
            ctx.existing_pairs.append(par)
        else:
            continue


@ctx_inject
def clear_ctx(ctx):
    ctx.existing_pairs = None


UNIQUE_BY = 'gender'


def reduce(factors, collection):
    global UNIQUE_BY
    if UNIQUE_BY in list(factors):
        yield collection


clear_ctx()
list(flatten(list(map(reduce, list(
    map(map_phase_1, input_collection)), input_collection))))

[User(id=0, age=55, social_contacts=20, gender='male'),
 User(id=1, age=25, social_contacts=240, gender='female')]

## Операторы реляционной алгебры

### Selection (Выборка)

**The Map Function**: Для каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.


In [10]:
def C(x): return True if x >= 0 else False


input_vals = [randint(-100, 100) for _ in range(ARR_SIZE)]


def MAP(pred, t):
    if pred:
        yield {t, t}


list(flatten(map(lambda x: x, map(MAP, map(C, input_vals), input_vals))))

[{80},
 {8},
 {78},
 {97},
 {10},
 {16},
 {26},
 {31},
 {3},
 {75},
 {3},
 {33},
 {31},
 {93},
 {94},
 {25},
 {3},
 {80},
 {25},
 {59},
 {29},
 {62},
 {9},
 {0},
 {20},
 {76},
 {0},
 {4},
 {6},
 {80},
 {4},
 {76},
 {93},
 {58},
 {67},
 {39},
 {33},
 {75},
 {25},
 {13},
 {11},
 {75},
 {25},
 {90},
 {96},
 {20},
 {11}]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.


In [11]:
from collections import OrderedDict

input_vals = [(randint(-100, 100), ) for _ in range(99)]


@ctx_inject
def reduce(ctx, x: dict[int, int]):
    if ctx.union is None:
        ctx.union = {}
        ctx.last_key = None
    ctx.union: dict
    if len(ctx.union.keys()) >= 1:
        if len(ctx.union[ctx.last_key]) == 1:
            for val in x.values():
                ctx.union[ctx.last_key].append(val)
        else:
            for val in x.values():
                ctx.union[val] = [val]
                ctx.last_key = val
    else:
        for val in x.values():
            ctx.union[val] = [val]
            ctx.last_key = val
    return ctx.union


@ctx_inject
def clear_ctx(ctx):
    ctx.union = None
    ctx.last_key = None


clear_ctx()

list(map(reduce, map(lambda x: {x[0]: x[0]}, input_vals)))[-1]

{-87: [-87, -75],
 -46: [-46, 49],
 57: [57, -52],
 91: [91, -3],
 -74: [-74, -55],
 -44: [-44, -12],
 -90: [-90, 81],
 -11: [-11, 38],
 33: [33, -12],
 52: [52, -35],
 79: [79, 49],
 -99: [-99, -10],
 12: [12, -52],
 -98: [-98, -69],
 -50: [-50, 33],
 30: [30, 60],
 -64: [-64, 98],
 96: [96, -96],
 -7: [-7, -78],
 -100: [-100, -100],
 -57: [-57, -11],
 47: [47, -54],
 66: [66],
 22: [22, 71],
 38: [38, 16],
 -45: [-45, -59],
 -20: [-20, 66],
 73: [73, -58],
 -31: [-31, 29],
 -6: [-6, -92],
 -79: [-79, -17],
 -40: [-40, 39],
 8: [8, 22],
 -21: [-21, 64],
 11: [11, 8],
 -30: [-30, 23],
 26: [26, 79],
 -4: [-4, 32],
 35: [35, 9],
 90: [90, -58],
 -70: [-70, 46],
 64: [64, 64],
 -58: [-58, -72]}

## Вычисление TF-IDF (Term Frequency – Inverse Document Fraquency)

Реализуется в три этапа:

**Этап 1:** Частота слова в документе

**Этап 2:** Количество документов, в которых встречается слово

**Этап 3:** Расчёт TF-IDF


In [12]:
DOC1 = """
Lorem ipsum dolor sit amet consectetur adipisicing elit. Dignissimos labore placeat ut saepe dolorum quisquam sint dolore distinctio vel ratione tempora perspiciatis iusto voluptates quos laudantium at aut recusandae nulla, officia molestias. Quidem commodi rerum itaque placeat recusandae illo. Quo nemo et repellendus in maxime beatae consequatur ullam doloremque debitis earum, quidem aliquid quasi dolorum enim similique id ipsam minus eum corrupti alias cum? Recusandae incidunt nostrum consequatur at sit repudiandae asperiores ab esse ut quos dolore optio debitis doloremque dolorem eum accusamus dolores eos minus, corporis ducimus quasi! Dolorem, sequi et optio aperiam vel ipsam suscipit! Delectus, eos sit!
"""
DOC2 = """
There are many variations of passages of Lorem Ipsum available, but the majority have suffered alteration in some form, by injected humour, or randomised words which don't look even slightly believable. If you are going to use a passage of Lorem Ipsum, you need to be sure there isn't anything embarrassing hidden in the middle of text. All the Lorem Ipsum generators on the Internet tend to repeat predefined chunks as necessary, making this the first true generator on the Internet. It uses a dictionary of over 200 Latin words, combined with a handful of model sentence structures, to generate Lorem Ipsum which looks reasonable. The generated Lorem Ipsum is therefore always free from repetition, injected humour, or non-characteristic words etc.
"""
DOC3 = """
It is a long established fact that a reader will be distracted by the readable content of a page when looking at its layout. The point of using Lorem Ipsum is that it has a more-or-less normal distribution distribution of letters
"""

docs = ['DOC1', 'DOC2', 'DOC3']
contents = [DOC1, DOC2, DOC3]

In [13]:
import math


@ctx_inject
def clear_ctx(ctx):
    ctx.docname = None
    ctx.doc_tn = {}
    ctx.sn = {}
    ctx.tf = {}


clear_ctx()


@ctx_inject
def MAP_1(ctx, docname: str, contentns: str):
    ctx.sn[docname] = 0

    def prepare_word(word: str):
        new_word = word.replace('\n', '')
        for symb in r"""!"#$%&()*+,-./:;=?@[]^_{}~""".split():
            new_word.replace(symb, '')
        return new_word
    for word in contentns.split(' '):
        ctx.sn[docname] += 1
        yield ((prepare_word(word,), docname), 1)


@ctx_inject
def REDUCE_1(ctx, par):
    for p in par:
        wd, cnt = p
        word, doc = wd
        if ctx.docname is None:
            ctx.docname = doc
        if doc != ctx.docname:
            for word, tn in ctx.doc_tn.items():
                tf = tn / ctx.sn[ctx.docname]
                yield ((word, ctx.docname), tf)
            ctx.docname = doc
        dtnc = ctx.doc_tn.get(word, 0) + cnt
        ctx.doc_tn[word] = dtnc

    for word, tn in ctx.doc_tn.items():
        tf = tn / ctx.sn[ctx.docname]
        yield ((word, ctx.docname), tf)


job_1 = map(lambda x: x, (map(REDUCE_1, map(MAP_1, docs, contents))))


def MAP_2(inp):
    ret = {}
    for obj in inp:
        wd, tf = obj
        word, doc = wd
        ret[word] = (doc, tf, 1)
    for word, meta in ret.items():
        yield (word, (meta))


@ctx_inject
def REDUCE_2(ctx, inp):
    docname = None
    for d in inp:
        word, dti = d
        doc, tf, cnt = dti
        docname = doc
        _, cur_n = ctx.tf.get((word), [tf, 0])
        cur_n += cnt
        ctx.tf[(word)] = [tf, cur_n]
    for word, tf_cnt in ctx.tf.items():
        yield (word, docname), (*tf_cnt,)


job_2 = map(REDUCE_2, map(MAP_2, job_1))


def MAP_3(inp):
    for word_doc, tf_n in inp:
        word, doc = word_doc
        tf, n = tf_n
        yield ((word, doc), tf * math.log10(len(docs) / n))


job_3 = list(map(MAP_3, list(job_2)))
list(flatten(list(job_3)))

[(('Lorem', 'DOC1'), 0.004771212547196624),
 (('ipsum', 'DOC1'), 0.004771212547196624),
 (('dolor', 'DOC1'), 0.004771212547196624),
 (('sit', 'DOC1'), 0.009542425094393249),
 (('amet', 'DOC1'), 0.004771212547196624),
 (('consectetur', 'DOC1'), 0.004771212547196624),
 (('adipisicing', 'DOC1'), 0.004771212547196624),
 (('elit.', 'DOC1'), 0.004771212547196624),
 (('Dignissimos', 'DOC1'), 0.004771212547196624),
 (('labore', 'DOC1'), 0.004771212547196624),
 (('placeat', 'DOC1'), 0.009542425094393249),
 (('ut', 'DOC1'), 0.009542425094393249),
 (('saepe', 'DOC1'), 0.004771212547196624),
 (('dolorum', 'DOC1'), 0.009542425094393249),
 (('quisquam', 'DOC1'), 0.004771212547196624),
 (('sint', 'DOC1'), 0.004771212547196624),
 (('dolore', 'DOC1'), 0.009542425094393249),
 (('distinctio', 'DOC1'), 0.004771212547196624),
 (('vel', 'DOC1'), 0.009542425094393249),
 (('ratione', 'DOC1'), 0.004771212547196624),
 (('tempora', 'DOC1'), 0.004771212547196624),
 (('perspiciatis', 'DOC1'), 0.004771212547196624)